# COGS 108 - Final Project


## Data Cleaning/Pre-processing

The first step in our project is to import our dependencies

In [86]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from collections import deque
from pytrends.request import TrendReq
from datetime import datetime
import time

In [87]:
import sys
!{sys.executable} -m pip install pytrends

You are using pip version 9.0.1, however version 9.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


First, we will define a helper function that we will use to return the celebrity name and death date, along with any other features we would like to extract. 

If we pass in any page that isn't a wikipedia page of a deceased notable celebrity (as per our definition), the function will return `None`.

In [2]:
def get_death_date(soup):
    table_selector = soup.select('table.infobox.vcard')

    if table_selector:
        table = table_selector[0]
        
        # Important features we would like to handle specially
        name = table.find('span', class_='fn')
        born = table.find('span', class_='bday')
        died = table.find('span', class_='dday deathdate')
        
        # Only return if the person has a name, has born data, and died
        if name and born and died:
            data =  {'Name': name.text, 'Born': born.text, 'Died': died.text}
            
            # Extract other features that we would like to keep.
            rows = table.find_all('tr')
            for row in rows:
                header = row.find('th')
                datum = row.find('td')
                if header and datum and header.text not in data:
                    data[header.text.replace('\xa0', ' ').strip()] = datum.get_text(' ', strip=True)
                    
            return data
        
    return None

Then, we will recursively traverse the links through Breadth First Search and visit all Wikipedia links reachable from the 'List of Celebrities' page. 

In [3]:
# Breadth First Search traversal.

wikipedia_url = 'https://en.wikipedia.org'
starting_endpoint = '/wiki/Lists_of_celebrities'
click_limit = 2

# Dictionary of visited links
# 'url' -> number of clicks taken to get to link
links_visited = {starting_endpoint: 0} 
# Queue of links to visit
links_queue = deque()
links_queue.appendleft(starting_endpoint)

# Create a buffer of dataframes
data_buffer = deque()

# Debug info
num_visited = 0

# Find all links on the page that lead to another wikipedia page.
while links_queue:
    link = links_queue.pop()
    page = requests.get(wikipedia_url + link)
    soup = BeautifulSoup(page.content, 'html.parser')
    num_visited += 1
    
    # Debug info
    if links_visited[link] >= click_limit - 1:
        num_links = len(links_visited)
        num_deads = len(data_buffer)
        percentage = format(100.0*(float(num_visited)/num_links), '.2f')
        print((percentage + "% - " + str(len(links_queue)) + " left, " + str(num_deads) + " recorded.").ljust(60),  end='\r', flush=True)
    else:
        print("Loading initial pages...", end='\r', flush=True)
    
    # See if page is of deceased person - if so, get the data
    results = get_death_date(soup)
    
    # Append to dataframe buffer if we've found a match
    if results:
        data_buffer.append(pd.DataFrame([results]))
    
    # Break early on click limit or if page isn't a list
    if links_visited[link] == click_limit or ("List" not in link and "Category:" not in link):
        continue
    
    content = soup.find('div', id='content')
    
    # Continue BFS on links in the page
    if content:
        for candidate in content.findAll('a'):
            if 'href' in candidate.attrs:
                link_in_page = urlparse(candidate.attrs['href']).path
                                
                # Add to queue if it's a link we haven't seen before.
                if link_in_page not in links_visited and link_in_page.startswith('/wiki/') and 'File' not in link_in_page:
                    links_visited[link_in_page] = links_visited[link] + 1
                    links_queue.appendleft(link_in_page)

# Concatenate all of the buffer into the dataframe that holds our celebrity data
df_celebrities = pd.concat(data_buffer)

Since this takes such a long time, we will run this once and cache it into a JSON file. All subsequent operations will be run off of the loaded JSON file.

In [5]:
with open('celebrities.json', 'w') as file:
    file.write(df_celebrities.to_json(orient='records'))

Now, we can read the file back in as the JSON in order to circumvent the crawling process for future usage.

In [88]:
df_celebrities = pd.read_json("celebrities.json", orient="records")

In [89]:
date_objects = []

base_date = datetime.strptime("2004-01-01", "%Y-%m-%d")

for date in df_celebrities["Died"]:
    date_objects.append(datetime.strptime(date, "%Y-%m-%d"))
df_celebrities["Died"] = date_objects


df_valid = df_celebrities[df_celebrities["Died"] >= base_date]

Since pytrends records begin in 2004, we will discard any deaths that happened before that. This leaves us with 846 celebrities that died from 2004-01-01 to present.

In [90]:
#Search word to compare it to
comparator = "Grand Canyon"

#Open the pytrends request
pytrends = TrendReq(hl='en-US', tz=360)

#Create a list for the new column in the table
trends = []

#Find the relative trend at time of death compared to comparator
#Comparator relative trend value taken from 2004-01-01
def relative_trend(name, died):
    #Enter the name for searching the trend
    kw_list = [comparator, name]
    #Get a dataframe of the relative trends
    e_catcher = 0
    while e_catcher == 0:
        try:
            pytrends.build_payload(kw_list, timeframe='all')
            df_py = pytrends.interest_over_time()
            e_catcher = 1
        except:
            time.sleep(3600)
    comparator_value = df_py[comparator].iloc[0]
    return df_py[(df_py.index.month == died.month) & (df_py.index.year == died.year)][name].iloc[0] - comparator_value
    

#Loop through and find all the trends relative to grand canyon
counter = 0
for name, died in zip(df_valid["Name"], df_valid["Died"]):
    #Check if there are more than two parts to the name
    name_split = name.split(" ")
    name_split_len = len(name_split)
    shortened_name = name_split[0] + " " + name_split[name_split_len-1]
    if  name_split_len > 2:
        long_rt = relative_trend(name,died)
        short_rt = relative_trend(shortened_name,died)
        trends.append(max(long_rt, short_rt))
    else:
        trends.append(relative_trend(name,died))
    counter += 1
    print(("Celebrity trends parsed = " + str(counter)), end='\r', flush=True)
df_valid['Trends'] = trends

/Users/joseph/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [99]:
with open('valid.json', 'w') as file:
    file.write(df_valid.to_json(orient='records'))